- fold 5, 1 ensemble
- cancel any fe
- extract train oof

In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [2]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from umap import UMAP
from sklearn import preprocessing
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold
from torch.nn.modules.loss import _WeightedLoss

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [3]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
drug = pd.read_csv(DATA_DIR + 'train_drug.csv')

In [4]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[~train.index.isin(noncons_train_index)].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [5]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [6]:
# https://www.kaggle.com/c/lish-moa/discussion/195195
NB_SPLITS = 5
seed = 14

folds = []
    
# LOAD FILES
train_score = targets.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = train_score.drug_id.value_counts()
vc1 = vc.loc[vc <= 19].index.sort_values()
vc2 = vc.loc[vc > 19].index.sort_values()
    
# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, shuffle = True, random_state = seed)
tmp = train_score.groupby('drug_id')[target_feats].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_feats])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, shuffle = True, random_state = seed)
tmp = train_score.loc[train_score.drug_id.isin(vc2)].reset_index(drop = True)
for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_feats])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
train_score['fold'] = train_score.drug_id.map(dct1)
train_score.loc[train_score.fold.isna(),'fold'] = train_score.loc[train_score.fold.isna(),'sig_id'].map(dct2)
train_score.fold = train_score.fold.astype('int8')
folds.append(train_score.fold.values)
    
np.array(folds)

array([[4, 4, 1, ..., 0, 1, 4]], dtype=int8)

# feature engineering

In [7]:
#X = train[g_feats+c_feats].copy().values
#select = VarianceThreshold(threshold=0.7)
#X_new = select.fit_transform(X)
#drop_feats = list(np.array(train[g_feats+c_feats].columns)[select.get_support()==False])
#len(drop_feats)

#train.drop(drop_feats, axis=1, inplace=True)
#test.drop(drop_feats, axis=1, inplace=True)

#g_feats = [i for i in train.columns if "g-" in i]
#c_feats = [i for i in train.columns if "c-" in i]

In [8]:
#for i in c_feats + g_feats:
    #ss = preprocessing.QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
#    ss = preprocessing.RobustScaler()
#    ss.fit(train[i].values.reshape(-1,1))
#    train[i] = ss.transform(train[i].values.reshape(-1,1))
#    test[i] = ss.transform(test[i].values.reshape(-1,1))

In [9]:
#c_num = 1
#pca_c_cols = ["pca-c"+str(i+1) for i in range(c_num)]
#pca = PCA(n_components=c_num,random_state=42)
#c_train = pca.fit_transform(train[c_feats])
#c_test = pca.transform(test[c_feats])
#c_train = pd.DataFrame(c_train, columns=pca_c_cols)
#c_test = pd.DataFrame(c_test, columns=pca_c_cols)

#g_num = 10
#pca_g_cols = ["pca-g"+str(i+1) for i in range(g_num)]
#pca = PCA(n_components=g_num, random_state=42)
#g_train = pca.fit_transform(train[g_feats])
#g_test = pca.transform(test[g_feats])
#g_train = pd.DataFrame(g_train, columns=pca_g_cols)
#g_test = pd.DataFrame(g_test, columns=pca_g_cols)

#train = pd.concat([train, c_train],axis=1)
#test = pd.concat([test, c_test],axis=1)
#train = pd.concat([train, g_train],axis=1)
#test = pd.concat([test, g_test],axis=1)

In [10]:
#uc_num = 1
#um = UMAP(n_neighbors=30, n_components=uc_num, random_state=42)
#um_c_cols = ["um-c"+str(i+1) for i in range(uc_num)]#
#uc_train = um.fit_transform(train[c_feats])
#uc_test = um.transform(test[c_feats])
#uc_train = pd.DataFrame(uc_train, columns=um_c_cols)
#uc_test = pd.DataFrame(uc_test, columns=um_c_cols)

#ug_num = 10
#um = UMAP(n_neighbors=30, n_components=ug_num, random_state=42)
#um_g_cols = ["um-g"+str(i+1) for i in range(ug_num)]
#ug_train = um.fit_transform(train[g_feats])
#ug_test = um.transform(test[g_feats])
#ug_train = pd.DataFrame(ug_train, columns=um_g_cols)
#ug_test = pd.DataFrame(ug_test, columns=um_g_cols)

#train = pd.concat([train, uc_train],axis=1)
#test = pd.concat([test, uc_test],axis=1)
#train = pd.concat([train, ug_train],axis=1)
#test = pd.concat([test, ug_test],axis=1)

In [11]:
#n_clusters_g = 15
#n_clusters_c = 5
#def create_cluster(train, test, kind, n_clusters):
#    if kind == "g":
#        train_ = train[pca_g_cols].copy()
#        test_ = test[pca_g_cols].copy()
#    else:
#        train_ = train[pca_c_cols].copy()
#        test_ = test[pca_c_cols].copy()    
#    kmeans = KMeans(n_clusters = n_clusters, random_state = 0).fit(train_)
#    train[f'clusters_{kind}'] = kmeans.labels_
#    test[f'clusters_{kind}'] = kmeans.predict(test_)
#    train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
#    test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
#    return train, test
    
#train, test = create_cluster(train, test, kind = 'g', n_clusters = n_clusters_g)
#train, test = create_cluster(train, test, kind = 'c', n_clusters = n_clusters_c)

In [12]:
def fe(df):
    tmp = df.copy()
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})      
    tmp.drop(["sig_id", "cp_type"], axis=1, inplace=True)
    return tmp

train = fe(train)
test = fe(test)

print(train.shape, test.shape)

(21948, 874) (3982, 874)


In [13]:
train["fold"] = np.array(folds).reshape(-1,1)

In [14]:
fn_train = train.copy().to_numpy()
fn_test = test.copy().to_numpy()
    
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()

In [15]:
fn_train.shape, fn_test.shape, fn_targets.shape

((21948, 875), (3982, 874), (21948, 206))

# modelling

In [16]:
class LogitsLogLoss(Metric):
    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):

        logits = 1 / (1 + np.exp(-y_pred))
        
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [17]:
MAX_EPOCH=200
device = "cuda" if torch.cuda.is_available() else "cpu"

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def modelling_tabnet(tr, target, te, sample_seed):
    seed_everything(sample_seed) 
    tabnet_params = dict(n_d=12, n_a=12, n_steps=1, gamma=1.3, seed = sample_seed,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
    test_cv_preds = []

    oof_preds = np.zeros([len(tr),target.shape[1]])
    scores = []
    for fold_nb in range(NB_SPLITS):
        print("FOLDS : ", fold_nb+1)
        
        ## model
        val_idx = tr[:,-1] == fold_nb
        train_idx = tr[:,-1] != fold_nb
        X_train, y_train = tr[train_idx, :], target[train_idx, :]
        X_val, y_val = tr[val_idx, :], target[val_idx, :]
        X_train = np.delete(X_train, -1, 1)
        X_val = np.delete(X_val, -1, 1)
        model = TabNetRegressor(**tabnet_params)
    
        model.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_val, y_val)],
              eval_name = ["val"],
              eval_metric = ["logits_ll"],
              max_epochs=MAX_EPOCH,
              patience=20, batch_size=1024, virtual_batch_size=128,
              num_workers=1, drop_last=False,
              # use binary cross entropy as this is not a regression problem
              loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
    
        preds_val = model.predict(X_val)
        # Apply sigmoid to the predictions
        preds =  1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logits_ll"])
        oof_preds[val_idx,:] = preds
        scores.append(score)

        # preds on test
        preds_test = model.predict(te)
        test_cv_preds.append(1 / (1 + np.exp(-preds_test)))
        
    test_preds_all = np.stack(test_cv_preds)
    print("OOF log loss:", log_loss(np.ravel(target), np.ravel(np.array(oof_preds))))
    aucs = []
    for task_id in range(206):
        aucs.append(roc_auc_score(y_true=target[:, task_id],y_score=oof_preds[:, task_id]))
    print(f"Overall AUC : {np.mean(aucs)}")
    return oof_preds, test_preds_all

target_oof = np.zeros([len(fn_train),fn_targets.shape[1]])
target_pred = np.zeros([len(fn_test),fn_targets.shape[1]])

seeds = [0]
for seed_ in seeds:
    oof_preds, test_preds_all = modelling_tabnet(fn_train, fn_targets, fn_test, seed_)
    target_oof += oof_preds / len(seeds)
    target_pred += test_preds_all.mean(axis=0) / len(seeds)

FOLDS :  1
Device used : cuda
epoch 0  | loss: 0.56822 | val_logits_ll: 0.32031 |  0:00:01s
epoch 10 | loss: 0.02073 | val_logits_ll: 0.02057 |  0:00:11s
epoch 20 | loss: 0.01911 | val_logits_ll: 0.01923 |  0:00:22s
epoch 30 | loss: 0.01795 | val_logits_ll: 0.01885 |  0:00:32s
epoch 40 | loss: 0.01726 | val_logits_ll: 0.01974 |  0:00:44s
epoch 50 | loss: 0.01696 | val_logits_ll: 0.01791 |  0:00:55s
epoch 60 | loss: 0.01648 | val_logits_ll: 0.01779 |  0:01:05s
epoch 70 | loss: 0.01624 | val_logits_ll: 0.01918 |  0:01:15s
epoch 80 | loss: 0.01608 | val_logits_ll: 0.01816 |  0:01:26s
epoch 90 | loss: 0.01591 | val_logits_ll: 0.01762 |  0:01:38s

Early stopping occured at epoch 92 with best_epoch = 72 and best_val_logits_ll = 0.01751
Best weights from best epoch are automatically used!
FOLDS :  2
Device used : cuda
epoch 0  | loss: 0.56335 | val_logits_ll: 0.30359 |  0:00:01s
epoch 10 | loss: 0.02052 | val_logits_ll: 0.02025 |  0:00:11s
epoch 20 | loss: 0.01904 | val_logits_ll: 0.01894 |  

In [18]:
aucs = []
for task_id in range(targets.shape[1]-1):
    aucs.append(roc_auc_score(y_true=targets.iloc[:, task_id+1].values,
                              y_score=target_oof[:, task_id]))
print(f"Overall AUC : {np.mean(aucs)}")

Overall AUC : 0.6175376498318935


In [19]:
p_min = 0.00005
p_max = 1 - p_min

t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0

t.drop("sig_id", axis=1, inplace=True)
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.clip(np.array(train_checkscore.iloc[:,1:]),p_min, p_max))))

OOF log loss:  0.016317946358678424
OOF log loss:  0.01632044573911291


In [20]:
train_checkscore.to_csv("tab_newval_oof.csv", index=False)

In [21]:
sub[target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)